In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

import shared_utils
import branca
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0418 18:20:33.881122632    1286 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0418 18:20:36.418128613    1286 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
%%sql -o rt_coverage

SELECT calitp_itp_id, calitp_url_number, MIN(vehicle.timestamp) as min_timestamp, MAX(vehicle.timestamp) as max_timestamp
FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
GROUP BY calitp_itp_id, calitp_url_number
ORDER BY max_timestamp DESC

,calitp_itp_id,calitp_url_number,min_timestamp,max_timestamp
0,474,0,1646508196,1650239985928
1,260,0,1645029415,1650239989
2,282,1,1644192008,1650239989
3,360,0,1644017957,1650239989
4,380,0,1420778367,1650239989
...,...,...,...,...
66,238,0,1646508196,1648681037
67,203,0,1644191966,1648559079
68,122,0,1646662923,1646851578
69,346,0,1644122957,1646175977


In [8]:
rt_coverage >> filter(_.max_timestamp > 1650249989)

,calitp_itp_id,calitp_url_number,min_timestamp,max_timestamp
0,474,0,1646508196,1650239985928


In [9]:
rt_filtered = rt_coverage >> filter(_.max_timestamp <= 1650249989)

In [10]:
## set system time
os.environ['TZ'] = 'America/Los_Angeles'
time.tzset()
time.tzname

('PST', 'PDT')

In [11]:
rt_filtered.max_timestamp = rt_filtered.max_timestamp.apply(convert_ts)
rt_filtered.min_timestamp = rt_filtered.min_timestamp.apply(convert_ts)

/tmp/ipykernel_1286/1618365186.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1286/1618365186.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
rt_filtered

,calitp_itp_id,calitp_url_number,min_timestamp,max_timestamp
1,260,0,2022-02-16 08:36:55,2022-04-17 16:59:49
2,282,1,2022-02-06 16:00:08,2022-04-17 16:59:49
3,360,0,2022-02-04 15:39:17,2022-04-17 16:59:49
4,380,0,2015-01-08 20:39:27,2022-04-17 16:59:49
5,350,0,2022-02-06 16:00:09,2022-04-17 16:59:49
...,...,...,...,...
66,238,0,2022-03-05 11:23:16,2022-03-30 15:57:17
67,203,0,2022-02-06 15:59:26,2022-03-29 06:04:39
68,122,0,2022-03-07 06:22:03,2022-03-09 10:46:18
69,346,0,2022-02-05 20:49:17,2022-03-01 15:06:17


https://airtable.com/shrXjIHW4f4Bk2PSG

In [15]:
air_view = pd.read_csv('airtable_rt_status.csv').rename(columns={'ITP ID': 'calitp_itp_id', 'GTFS Realtime Status': 'rt_status'})
air_view.calitp_itp_id = air_view.calitp_itp_id.fillna(9999).astype(int)

In [16]:
air_view.columns

Index(['Name', 'calitp_itp_id', 'Reporting Category', 'Caltrans District',
       'GTFS Static Status', 'rt_status', 'Missing Static',
       'Services Needing Alerts',
       'Services Needing TripUpdates or VehiclePositions',
       'Immediate GTFS Goals'],
      dtype='object')

In [17]:
air_view.shape

(186, 10)

In [18]:
air_joined = air_view >> inner_join(_, rt_filtered, on = 'calitp_itp_id') >> distinct(_.calitp_itp_id, _keep_all=True)

In [24]:
air_joined.to_parquet('airtable_joined.parquet')

In [20]:
# air_joined.to_csv('airtable_joined.csv')

In [21]:
air_joined.shape

(52, 13)

In [22]:
air_joined

,Name,calitp_itp_id,Reporting Category,Caltrans District,GTFS Static Status,rt_status,Missing Static,Services Needing Alerts,Services Needing TripUpdates or VehiclePositions,Immediate GTFS Goals,calitp_url_number,min_timestamp,max_timestamp
0,Stanislaus Regional Transit Authority,484,Core,NaN,Static Incomplete,RT Incomplete,"Stanislaus Regional Transit, StaRT Commuter, S...","Stanislaus Regional Transit, StaRT Commuter, S...","Stanislaus Regional Transit, StaRT Commuter, S...","0 - Does not have GTFS Schedule,0 - Does not h...",0,2022-04-06 14:57:53,2022-04-17 16:59:37
1,Ventura County Transportation Commission,380,Core,07 - Los Angeles,Static OK,RT OK,NaN,Valley Express Dial-A-Ride,Valley Express Dial-A-Ride,"3 - GTFS data has zero validation errors,3 - G...",0,2015-01-08 20:39:27,2022-04-17 16:59:49
2,City of Santa Monica,300,Core,07 - Los Angeles,Static OK,RT OK,NaN,"Mobility on Demand Everyday, Santa Monica Coll...","Mobility on Demand Everyday, Santa Monica Coll...","2 - GTFS data has active service,4 - GTFS data...",0,2022-02-06 15:55:20,2022-04-17 16:59:26
3,City of Los Angeles,183,Core,07 - Los Angeles,Static OK,RT OK,Cityride Connector,"LAnow, LA Now, Cityride Dial-A-Ride, Cityride ...","LAnow, LA Now, Cityride Dial-A-Ride, Cityride ...","2 - GTFS data has active service,2 - GTFS data...",0,2014-12-31 16:08:37,2022-04-17 16:59:49
4,Los Angeles County Metropolitan Transportation...,182,Core,07 - Los Angeles,Static OK,RT OK,NaN,LA Metro Micro,LA Metro Micro,"3 - GTFS data has zero validation errors,3 - G...",1,2022-02-06 15:58:17,2022-04-17 16:59:46
5,City of Pasadena,243,Core,07 - Los Angeles,Static OK,RT OK,NaN,Pasadena Dial-A-Ride,Pasadena Dial-A-Ride,"4 - GTFS data has zero validation warnings,4 -...",0,2022-02-06 15:57:19,2022-04-17 16:59:28
6,City of Santa Clarita,295,Core,07 - Los Angeles,Static OK,RT OK,NaN,"Go! Santa Clarita, Santa Clarita Summer Trolly...","Go! Santa Clarita, Santa Clarita Summer Trolly...","3 - GTFS data has zero validation errors,4 - G...",0,2022-02-06 15:59:20,2022-04-17 16:59:28
7,Long Beach Transit,170,Core,07 - Los Angeles,Static Incomplete,RT Incomplete,AquaLink,"Galaxy Express, AquaLink, Long Beach Dial-A-Lift","Galaxy Express, AquaLink, Long Beach Dial-A-Lift","2 - GTFS data has active service,0 - Does not ...",0,2022-02-06 15:54:38,2022-04-17 16:59:08
8,City of Burbank,45,Core,07 - Los Angeles,Static OK,RT OK,NaN,Burbank Bus Senior and Disabled,Burbank Bus Senior and Disabled,"2 - GTFS data has active service,4 - GTFS data...",0,2015-01-04 23:08:07,2022-04-15 20:41:47
9,City of Redondo Beach,260,Core,07 - Los Angeles,Static OK,RT OK,NaN,WAVE Dial-A-Ride,WAVE Dial-A-Ride,"4 - GTFS data has zero validation warnings,4 -...",0,2022-02-16 08:36:55,2022-04-17 16:59:49


In [ ]:
rt_coverage >> inner_join(_, tbl.gtfs_schedule.agency() >> collect(), on = ['calitp_itp_id']) >> filter(_.calitp_itp_id == 182)